## Installs/imports

In [1]:
!pip install -U datasets sentence-transformers pinecone-client tqdm
!pip install gradio -q

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB

In [2]:
# Imports!
import gradio as gr
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import pinecone
from tqdm.auto import tqdm


#Model code

In [3]:
ytt = load_dataset(
    "pinecone/yt-transcriptions",
    split="train",
    revision="926a45"
)
ytt

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/pinecone___json/pinecone--yt-transcriptions-4383a838c5c147e8/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


Dataset({
    features: ['video_id', 'text', 'start_second', 'end_second', 'url', 'title', 'thumbnail'],
    num_rows: 11298
})

This piece of the code was to initially "instantiate" the dictionary for the pinecone index. It's been commented out as it doesn't need to run again after the online index was created.

In [ ]:
#Dataset({
#    features: ['video_id', 'text', 'start_second', 'end_second', 'url', 'title', 'thumbnail'],
#    num_rows: 11298
#})

In [4]:
for x in ytt:
    print(x)
    break

{'video_id': 'ZPewmEu7644', 'text': " hi this is Jeff Dean welcome to applications of deep neural networks of Washington University in this video we're going to look at how we can use ganz to generate additional training data for the latest on my a I course and projects click subscribe in the bell next to it to be notified of every new video Dan's have a wide array of uses beyond just the face generation that you", 'start_second': 0, 'end_second': 20, 'url': 'https://www.youtube.com/watch?v=ZPewmEu7644&t=0s', 'title': 'GANS for Semi-Supervised Learning in Keras (7.4)', 'thumbnail': 'https://i.ytimg.com/vi/ZPewmEu7644/maxresdefault.jpg'}


##Loading transformer model

In [5]:
retriever = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_mpnet-base')
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [6]:
embed_dim = retriever.get_sentence_embedding_dimension()
embed_dim

768

## Accessing pinecone API where we store our metadata

In [7]:
# get api key from app.pinecone.io
pinecone.init(
    api_key="2a7bc2dd-d010-4b38-87b2-e6f925e190ce",
    environment="us-east1-gcp"
)

# create index
#pinecone.create_index(
#    "youtube-search",
#    dimension=embed_dim,
#    metric="cosine"
#)

Here we instantiate the index variable from the pinecone API

In [8]:
# connect to new index
index = pinecone.Index("youtube-search")

This piece of code was for configuring the pinecone index, and so running it again will yield an error saying that the index already exists. Therefore it's been commented out to allow the notebook to run smoothly.

In [ ]:
# This FREDRIK run for loading all of the data to the pinecone index (DOESNT HAVE TO BE RUN AGAIN)

# docs = []  # this will store IDs, embeddings, and metadata

# batch_size = 64

#for i in tqdm(range(0, len(ytt), batch_size)):
#    i_end = min(i+batch_size, len(ytt))
    # extract batch from YT transactions data
#    batch = ytt[i:i_end]
    # encode batch of text
#    embeds = retriever.encode(batch['text']).tolist()
    # each snippet needs a unique ID
    # we will merge video ID and start_seconds for this
#    ids = [f"{x[0]}-{x[1]}" for x in zip(batch['video_id'], batch['start_second'])]
    # create metadata records
#    meta = [{
#        'video_id': x[0],
 #       'title': x[1],
  #      'text': x[2],
  #      'start_second': x[3],
  #      'end_second': x[4],
  #      'url': x[5],
  #      'thumbnail': x[6]
  #  } for x in zip(
  #      batch['video_id'],
  #      batch['title'],
  #     batch['text'],
  #      batch['start_second'],
  #      batch['end_second'],
   #     batch['url'],
   #     batch['thumbnail']
   # )]
    # create list of (IDs, vectors, metadata) to upsert
   # to_upsert = list(zip(ids, embeds, meta))
    # add to pinecone
   # index.upsert(vectors=to_upsert)
#index.describe_index_stats()

## Testing out query functionality

In [9]:
query = "What is deep learning?"

xq = retriever.encode(query).tolist()

In [10]:
xc = index.query(xq, top_k=5,
                 include_metadata=True)
for context in xc['matches']:
    print(context['metadata']['text'], end="\n---\n")
    print(context['metadata']['url'], end="\n---\n")

 terms of optimization but what's the algorithm for updating the parameters or updating whatever the state of the network is and then the the last part is the the data set like how do you actually represent the world as it comes into your machine learning system so I think of deep learning as telling us something about what does the model look like and basically to qualify as deep I
---
https://www.youtube.com/watch?v=Z6rxFNMGdn0&t=1154s
---
 any theoretical components any theoretical things that you need to understand about deep learning can be sick later for that link again just watched the word doc file again in that I mentioned the link also the second channel is my channel because deep learning might be complete deep learning playlist that I have created is completely in order okay to the other
---
https://www.youtube.com/watch?v=AuqZ4recf0s&t=591s
---
 under a rock for the last few years you have heard of the deep networks and how they have revolutionised computer vision and kind

#Gradio implementation

##Gradio version 1

This is our first implementation of Gradio, but it had some issues with the visual appeal. We used ChatGPT in order to help get around some of the issues, as we thought that it was more important to get this done quick so that we can play around with other models as well. The second version ended up being the functional one.

In [ ]:
#def sem_search(query):
#  xq = retriever.encode(query).tolist()
#  xc = index.query(xq, top_k=5,
#                 include_metadata=True)
#  for context in xc['matches']:
#    print(context['metadata']['text'], end="\n---\n")
#    print(context['metadata']['url'], end="\n---\n")
#  return {"text": context['metadata']['text'], "metadata_url": context['metadata']['url']}

In [ ]:
#sem_search('apple')

In [ ]:
#gr.Interface(fn=sem_search, inputs=["text"], outputs=["text"],examples_per_page=5).launch()

##Gradio version 2

So I cheekily asked ChatGPT for a bit of help. What it's essentially done here, is to adapt our code to create the variables text and url, which contains the corresponding metadata. It then create a formatted string called "results" where it throws these variables in with some breaklines for better visual formatting (makes it look pretty). If you compare to the original function, you can also see it instantiates an empty list called results, which it will append each formated string to, in order to display all 5 of them within gradio. Hence the results.append(result). Finally the function then returns the concatenated version of the results with "\n" to create breaklines within the display.

In [11]:
def sem_search(query):
  xq = retriever.encode(query).tolist()
  xc = index.query(xq, top_k=5, include_metadata=True)
  results = []
  for context in xc['matches']:
    text = context['metadata']['text']
    url = context['metadata']['url']
    result = f"{text}\n\n{url}\n\n{'-'*50}\n"
    results.append(result)
  return "\n".join(results)

In [12]:
gr.Interface(
    fn=sem_search, 
    inputs=["text"], 
    outputs="text",
    examples=[["Deep Learning"], ["Neural Networks"], ["Coding stuff"], ["More examples"], ["Too many examples?"], ["NO MORE EXAMPLES PLEASE LORD NO!"]],
    examples_per_page=6
).launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Quite frankly, the search results are really bad. Except a few search terms, it seems to not be able to semantically find relevant videos. This could in part be due to a relatively limited number of videos to search from, but also the underlying model not quite interpreting stuff correctly. There seems to be a bit of bias towards data-related subjects.